### MATH2504 S2, 2022, BigHW Submission 
### Submission form II of IV 

Student #1 name: Lief

Student #1 ID:

Student #2 name:

Student #2 ID:

GitHub repo: 

---

**Note: It is expected that this notebook be runnable in the sense that the marker can select "Restart & Run All" and all code cells of solutions will execute. This requires not to have name clashes between solutions.**

---

# Solution to [Question 5](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q5)

In [3]:
function get_factors(n)
    factors = [1]
    for i in 2:floor(sqrt(n))
        if n % i == 0
            push!(factors, i, n / i)
        end
    end
    return factors
end

perfects = [i for i in 1:10^6 if sum(get_factors(i))==i]

print("The first $(length(perfects)) perfect numbers are $(perfects)")

The first 5 perfect numbers are [1, 6, 28, 496, 8128]

# Solution to [Question 6](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q6)

In [5]:
using Plots
using StatsBase
using Profile
using ProfileView

function sieve_of_Eratosthenes(n)
    primebits = ones(Bool,n) #Will contain true if the index is prime (initially all assumed prime)
    primebits[1] = false #The number 1 is not prime
    p = 2 #Smallest prime
    while p ≤ n
        i = 2p
        while i ≤ n  # \le +[TAB]
            primebits[i] = false
            i += p
        end
        p += 1
        while p ≤ n && !primebits[p]
            p += 1
        end
    end
    (1:n)[primebits]
end

function run(n::Int64)
    even_range = 4:2:n
    primes ::Vector{Int64} = sieve_of_Eratosthenes(n)[2:end]
    #exclude 2, because odd sums are problematic for us
    pairs = zeros(Int64,2*length(even_range))
    pairs[1] = 1 #account for 2+2 = 4
    for (i, p) in enumerate(primes)
        for j in i:length(primes)
            sum = p + primes[j]
            pairs[sum÷2 - 1] += 1
        end
    end
    checks = pairs[1:length(even_range)]
    if 0 ∈ checks
        println("Found a counter example for Goldbach")
    end
    
    x = sample(even_range,min(2*10^4,n÷3),replace = false)
    y = checks[(-1 .+ x.÷2)]
    scatter(x,y,legend=false,xlabel="N",ylabel="Number of Goldbach pairs")
    png("scatter.png")
    return nothing
end

@time run(10^6)

  5.660514 seconds (786.03 k allocations: 33.744 MiB)


![](scatter.png)

# Solution to [Question 7](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q7)

## 7a:

In [6]:
### Answer 7a here (in place of this comment)

## 7b:

In [7]:
### Answer 7b here (in place of this comment)

## 7c:

In [8]:
### Answer 7c here (in place of this comment)

## 7d:

In [9]:
### Make this a markdown cell and answer 7d here (in place of this comment)

---

*Use this area below as an experimental playground for the question. Feel free to leave some of the experimental code here with your submision. However, it will not be marked or reviewed. Make sure in your submission that if there is any experimental code here then running it does not interfere with running the actual solutions.*

---

# Solution to [Question 8](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q8)

## 8a:

The code below has been modified to sort by length of last name:

In [2]:
function array_table(array, heading)
    println(heading,":")
    for (i,a) in enumerate(array)
        println(i,"\t",a)
    end
end

names_list = [   "Amy Chan",
            "Maithili Mehta",
            "Anna Foeglein",
            "Andy Ferris",
            "Thomas Graham",
            "Elaine Schenk",
            "Jesse Woods",
            "Tina Moghaddam",
            "Paul Bellette",
            "Paul Vrbik",
            "Tom Cranitch",
            "Yoni Nazarathy",
            "Sam Hambleton",
            "Alistair Falconer",
            "Emma Comino",
            "Ivana Carrizo-Molina"]

sorted_names = sort(names_list)
array_table(sorted_names, "Sorted by first name")

println()

sorted_names = sort(names_list,by=(x)->length(split(x," ")[2]))
array_table(sorted_names,"Sorted by length of last name")

Sorted by first name:
1	Alistair Falconer
2	Amy Chan
3	Andy Ferris
4	Anna Foeglein
5	Elaine Schenk
6	Emma Comino
7	Ivana Carrizo-Molina
8	Jesse Woods
9	Maithili Mehta
10	Paul Bellette
11	Paul Vrbik
12	Sam Hambleton
13	Thomas Graham
14	Tina Moghaddam
15	Tom Cranitch
16	Yoni Nazarathy



Sorted by length of last name:
1	Amy Chan
2	Maithili Mehta
3	Jesse Woods
4	Paul Vrbik
5	Andy Ferris
6	Thomas Graham
7	Elaine Schenk
8	Emma Comino
9	Anna Foeglein
10	Paul Bellette
11	Tom Cranitch
12	Alistair Falconer
13	Tina Moghaddam
14	Yoni Nazarathy
15	Sam Hambleton
16	Ivana Carrizo-Molina


## 8b:

In [14]:
function sort_bubble(list; by=(x)->x)
    for i in 1:length(list)-1
        for j in 1:length(list)-i
            if by(list[j]) > by(list[j+1])
                tmp = list[j]
                list[j] = list[j+1]
                list[j+1] = tmp
            end
        end
    end
    return list
end

sorted_names = sort_bubble(names_list,by=(x)->reverse(split(x," ")))
array_table(sorted_names,"Bubble sorted by last name")

Bubble sorted by last name:
1	Paul Bellette
2	Ivana Carrizo-Molina
3	Amy Chan
4	Emma Comino
5	Tom Cranitch
6	Alistair Falconer
7	Andy Ferris
8	Anna Foeglein
9	Thomas Graham
10	Sam Hambleton
11	Maithili Mehta
12	Tina Moghaddam
13	Yoni Nazarathy
14	Elaine Schenk
15	Paul Vrbik
16	Jesse Woods


## 8c:

In [4]:
#first implementation of quicksort, not in place
function quick_sort(list;by=(x)->x)
    if length(list)<=1
        return list
    end
    pivot = by(list[1]) #The choice of pivot can be optimised but this is sufficient for our purposes
    left = [x for x in list[2:end] if by(x)<pivot]
    right = [x for x in list[2:end] if by(x)>=pivot]
    return(cat(quick_sort(left,by=by),[list[1]],quick_sort(right,by=by),dims=1))
end 

#in place implementation, far more memory efficient
function quick_sort_in_place(list,lo=1,hi=length(list);by=(x)->x)
    if lo>=hi || lo < 1
        return list
    end
    pivot = by(list[lo])
    i = lo+1
    for j in lo+1:hi
        if by(list[j])<pivot
            list[i],list[j] = list[j],list[i]
            i += 1
        end
    end
    list[lo],list[i-1] = list[i-1],list[lo]
    quick_sort_in_place(list,lo,i-2,by=by)
    quick_sort_in_place(list,i,hi,by=by)
    return list
end

sorted_names = quick_sort_in_place(names_list,by=(x)->reverse(split(x," ")))
array_table(sorted_names,"Quick sorted by last name")

Quick sorted by last name:
1	Paul Bellette
2	Ivana Carrizo-Molina
3	Amy Chan
4	Emma Comino
5	Tom Cranitch
6	Alistair Falconer
7	Andy Ferris
8	Anna Foeglein
9	Thomas Graham
10	Sam Hambleton
11	Maithili Mehta
12	Tina Moghaddam
13	Yoni Nazarathy
14	Elaine Schenk
15	Paul Vrbik
16	Jesse Woods


## 8d:

In [6]:
using BenchmarkTools
function test(n)
    data = rand(Int,n)
    println("system sort")
    @btime sort($data)
    println("quick sort")
    @btime quick_sort_in_place($data)
    return nothing
end

test(10^4) #larger values take a long time to run because btime takes multiple samples


system sort


  549.958 μs (2 allocations: 78.17 KiB)
quick sort


  38.908 ms (0 allocations: 0 bytes)


Bubble sort has $O(n^2)$ time complexity, which means that the running time tends to grow quadratically with the size of the input. We can test with some smaller values and fit a quadratic:

![](bubble_sort.png)

Extrapolating this trend gives an estimated running time of 8.5 minutes for $10^6$ items, and 1400 hours for $10^8$ items, which is unreasonable to run.

We can however run trials for quick sort and the system sort. I obtained the following results:

$10^6$ items:

 - System sort = 89.285 ms
 - Quick sort = 132.084 ms

$10^7$ items:

 - System sort = 1.050 s
 - Quick sort = 2.107 s

$10^8$ items:

 - System sort = 12.082 s
 - Quick sort = 33.362 s

# Solution to [Question 9](https://courses.smp.uq.edu.au/MATH2504/2022/assessment_html/bighw.html#q9)

## 9a:

In [17]:
### Answer 9a here (in place of this comment)

## 9b:

In [18]:
### Answer 9b here (in place of this comment)

## 9c:

In [19]:
### Answer 9c here (in place of this comment)

## 9d:


In [20]:
### Make this a markdown cell and answer 9d here (in place of this comment)

---

*Use this area below as an experimental playground for the question. Feel free to leave some of the experimental code here with your submision. However, it will not be marked or reviewed. Make sure in your submission that if there is any experimental code here then running it does not interfere with running the actual solutions.*

---